In [ ]:
!pip install transformers datasets peft

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft.tuners.lora import LoraConfig, LoraModel

Load model and tokenizer

In [ ]:
model_checkpoint = "gpt-neo-2.7B"
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Define lambda regularization parameter as per paper details

In [ ]:
lambda_reg = 0.1

Placeholder for the dataset loading function

In [ ]:
dataset = [{"prompt": "Example prompt", "response": "Example response"}]

Define LoRA configuration

In [ ]:
lora_config = LoraConfig(
    r=128,  # rank of LoRA
    lora_alpha=256,  # scaling factor for initialization
    lora_dropout=0.05,
    bias="none",
)

Wrap the model with LoRA layers for parameter-efficient training

In [ ]:
peft_model = LoraModel(model, lora_config).to(device)

Define the compute_spin_loss function (unchanged from previous)

In [ ]:
def compute_spin_loss(model_logits_gt, opponent_logits_gt, model_logits_syn, opponent_logits_syn, lambda_reg):
    model_probs_gt = torch.nn.functional.softmax(model_logits_gt, dim=-1)
    model_probs_syn = torch.nn.functional.softmax(model_logits_syn, dim=-1)
    opponent_probs_gt = torch.nn.functional.softmax(opponent_logits_gt, dim=-1)
    opponent_probs_syn = torch.nn.functional.softmax(opponent_logits_syn, dim=-1)

    # Calculate losses
    loss_gt = -torch.log(model_probs_gt / opponent_probs_gt)
    loss_syn = -torch.log(model_probs_syn / opponent_probs_syn)

    # Apply the logistic loss to the log odds ratio
    logistic_loss_gt = torch.log(1 + torch.exp(-lambda_reg * loss_gt))
    logistic_loss_syn = torch.log(1 + torch.exp(-lambda_reg * loss_syn))

    # Combine losses for the final spin loss
    spin_loss = logistic_loss_gt.mean() + logistic_loss_syn.mean()
    return spin_loss

Training setup

In [ ]:
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, peft_model.parameters()), lr=5e-5)

Training loop for T iterations

In [ ]:
T = 5  # Set the number of iterations
for iteration in range(T):
    total_loss = 0
    
    # Disable adapter layers for the opponent model
    peft_model.disable_adapter_layers()
    
    synthetic_data = []
    for data in dataset:
        prompt = data['prompt']
        # Tokenize and generate synthetic data using the opponent model
        prompt_ids = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).input_ids.to(device)
        with torch.no_grad():
            peft_model.eval()  # Set model to evaluation mode
            synthetic_response_ids = peft_model.generate(prompt_ids, max_length=50)
            synthetic_data.append(synthetic_response_ids)
    
    # Enable adapter layers for training the main player model
    peft_model.enable_adapter_layers()
    
    # Train the main player model using the synthetic data and real responses
    peft_model.train()  # Set model to training mode
    for i, data in enumerate(dataset):
        ground_truth = data['response']
        ground_truth_ids = tokenizer(ground_truth, return_tensors='pt', padding=True, truncation=True).input_ids.to(device)
        synthetic_response_ids = synthetic_data[i]

        # Calculate logits for ground truth and synthetic responses
        main_player_logits_gt = peft_model(ground_truth_ids).logits
        main_player_logits_syn = peft_model(synthetic_response_ids).logits

        # Get opponent's logits for synthetic responses (as they were generated before enabling LoRA)
        with torch.no_grad():
            opponent_logits_syn = peft_model(synthetic_response_ids).logits
        
        # Compute the loss (assuming the function is defined above)
        loss = compute_spin_loss(
            main_player_logits_gt, opponent_logits_syn, 
            main_player_logits_syn, opponent_logits_syn, 
            lambda_reg
        )
        total_loss += loss.item()

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Print average loss
    average_loss = total_loss / len(dataset)
    print(f"Iteration {iteration + 1}/{T}, Average Loss: {average_loss}")

Save the final model parameters

In [ ]:
final_model_params = peft_model.state_dict()
print("Training complete.")